In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import transforms3d as t3d
import datetime
import torch
from torch.nn import functional as F

import sys
sys.path.append("../")
import curvvae_lib.train.predictive_passthrough_trainer as ppttrainer
import curvvae_lib.architecture.passthrough_vae as ptvae
import curvvae_lib.architecture.save_model as sm
import curvvae_lib.architecture.load_model as lm

In [2]:
foodname = "banana"
foldername = f"fork_trajectory_{foodname}"
savefilename = f"{foodname}_clean_pickups"

In [3]:
train = []
training_ts = np.linspace(0,1,64)
attempt = 1
while True:
    try:
        raw_vals = np.load(f"{savefilename}/pickup_attempt{attempt}.npy")
        train.append(raw_vals.T.flatten())
    except:
        print(f"We found {attempt-1} pickup attempts")
        break
    attempt += 1

train = np.array(train).reshape(-1,7,64)
all_points = train[:,:,:]

time_shape = list(all_points.shape)
time_shape[1] = 1
# why be smart when you can be dumb
t = np.ones(time_shape)
for i in range(time_shape[2]):
    t[:,:,i] = t[:,:,i] * i / (time_shape[2] + 0.0)

all_points = np.concatenate((t, all_points), axis=1)
all_points = all_points.transpose(0,2,1)
print(all_points.shape)

# See http://localhost:8889/notebooks/scratchwork/2021-09-17%20Rotation%20Scaling.ipynb
# for why we want quaternion values to be multiplied by 0.16 when position values are in meters 
# (if the relevant distance scale of the fork is 0.08 meters, ie: 8cm).
# scaling term doesn't affect time, so don't use time in calculation
stats_reshaped = all_points.reshape(-1,8)
mean = np.mean(stats_reshaped, axis=0)
mean[0] = 0 # don't scale time
variance = np.var(stats_reshaped[:,1:], axis=0) # don't scale time
print(mean)
print(variance)
position_std = np.sqrt(np.max(variance))
print("std of: ", position_std)
position_scaling = 1/position_std
rotation_scaling = 0.16 * position_scaling

We found 179 pickup attempts
(179, 64, 8)
[ 0.          0.0021176  -0.00108136  0.01226278 -0.05341325  0.3809648
 -0.86529588 -0.02303113]
[2.74856348e-05 3.50234479e-05 1.02086472e-04 1.03992603e-02
 5.06524753e-02 2.98499090e-02 1.16367323e-02]
std of:  0.2250610480383746


In [4]:
def scale_dataset(input_points):
    points = input_points - mean
    poss = position_scaling
    rts = rotation_scaling
    points = (points * np.array((1,poss,poss,poss,rts,rts,rts,rts)))
    return points
    
def unscale_dataset(input_points):
    poss = position_scaling
    rts = rotation_scaling
    points = (input_points / np.array((1,poss,poss,poss,rts,rts,rts,rts)))
    points = points + mean
    return points

In [5]:
scaled_points = scale_dataset(all_points)
print(np.mean(scaled_points.reshape(-1,8),axis=0))
print(np.var(scaled_points.reshape(-1,8),axis=0))

[ 4.92187500e-01 -2.81528009e-17 -2.17591502e-17  2.48869826e-16
 -5.98249433e-18 -2.07415750e-16 -3.70239233e-15 -2.10880351e-17]
[0.08331299 0.00054263 0.00069145 0.00201543 0.00525584 0.0256
 0.01508628 0.00588126]


In [6]:
def LoadDataBatch(all_points, batchsize, passthroughdim, predictive):
    """Sample Pair of Points from Trajectory"""
    # all_points should be of dimension: num_trajectories, numtimesteps, 1+spatialdims
    traj_ids = np.random.choice(all_points.shape[0], batchsize)
    t1_ids = np.random.choice(all_points.shape[1], batchsize)
    if predictive:
        t2_ids = np.random.choice(all_points.shape[1], batchsize)
    else:
        t2_ids = t1_ids
    return (torch.tensor(all_points[traj_ids, t1_ids,passthroughdim:], dtype=torch.float),
            torch.tensor(all_points[traj_ids, t1_ids,:passthroughdim], dtype=torch.float),
            torch.tensor(all_points[traj_ids, t2_ids,passthroughdim:], dtype=torch.float),
            torch.tensor(all_points[traj_ids, t2_ids,:passthroughdim], dtype=torch.float))

In [7]:
print(all_points.shape)

(179, 64, 8)


In [8]:
class Loader(object):
    def __init__(self, all_points, batchsize, passthroughdim, epochnumbatches, predictive):
        self.all_points = all_points
        self.batchsize = batchsize
        self.passthroughdim = passthroughdim
        self.epochnumbatches = epochnumbatches
        self.predictive = predictive
        
    def __iter__(self):
        self.n = 0
        return self
    
    def __next__(self):
        if self.n >= self.epochnumbatches:
            # https://docs.python.org/3/library/exceptions.html#StopIteration
            raise StopIteration
        self.n += 1
        return LoadDataBatch(self.all_points, self.batchsize, self.passthroughdim, self.predictive)
    

In [9]:

train_loader = Loader(scaled_points, 256, 1,100,predictive=True)
for allxs, allts, testx2, testt2 in train_loader:
    print(allts[:1],testt2[:1])
    break

tensor([[0.9219]]) tensor([[0.6719]])


In [10]:
def make_vae(latent_dim):
    input_dim = 7 
    passthrough_dim = 1
    emb_layer_widths = [1000]
    recon_layer_widths = [1000]
    dtype = torch.float
    model = ptvae.FCPassthroughVAE(input_dim, passthrough_dim, latent_dim,
        emb_layer_widths, recon_layer_widths, dtype)
    return model
allxs, allts, t2s, x2s = LoadDataBatch(scaled_points, 3000, 1, predictive=False)

In [11]:
testname = f"trainedmodels/{foodname}_"

In [14]:
all_exceptions = []

for lr in [0.0005,0.0001]:
    for curvreg in [0]:#[0.001,0,0.01,0.0001,0.0005]:
        for beta in [0.002,0.003,0.004,0.005]:#[0.001,0.00001,0.0001,0.01]:
            for latentdim in [3]:
                for epsilon_scale_start in [1]:
                    try:
                        vae = make_vae(latent_dim=latentdim)
                        device = "cpu"
                        num_epochs = 1000
                        savedir  = f'{testname}lat{latentdim}_curvreg{curvreg}_beta{beta}_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
                        print(savedir)
                        trainer = ppttrainer.PPTVAETrainer(vae, train_loader, beta=beta, device=device,
                            log_dir=savedir, lr=lr, annealingBatches=0, record_loss_every=300, loss_func = "gaussian")
                        scheduler = torch.optim.lr_scheduler.ExponentialLR(trainer.optimizer, gamma=0.5)
                        epoch = 0
                        epsilon_scale = epsilon_scale_start
                        for _ in range(num_epochs*3):
                            epoch += 1
                            # had an off-by-one error here before
                            # update values BEFORE training 501st element
                            if epoch % num_epochs == 1 and epoch > 1: 
                                scheduler.step()
                                epsilon_scale = epsilon_scale * np.sqrt(0.5)
                            embst = trainer.train(second_deriv_regularizer=0, curvature_regularizer=curvreg, epsilon_scale = epsilon_scale, num_new_samp_points=512)
                            is_error = embst[0]
                            if is_error:
                                raise Exception


                            if epoch % num_epochs == 0:
                                modeled = vae.noiseless_autoencode(allxs, allts)[0].detach().cpu().numpy()
                                print(np.sum(np.square(modeled - allxs.detach().cpu().numpy())))
                                print('====> Epoch: {}'.format(epoch))
                                sm.save_fcpassthrough_vae(vae, savedir+f"_epoch{epoch}")
                        sm.save_fcpassthrough_vae(vae, savedir)
                        print(savedir)
                    except Exception as e:
                        all_exceptions.append(e)

trainedmodels/banana_lat3_curvreg0_beta0.002_20221031-111739


KeyboardInterrupt: 